In [1]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          23988         370       12619        5024       10998       18317
Swap:             0           0           0


In [2]:
# Importación de librerías a utilizar
import os
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Leemos el dataset
df = pd.read_csv('../dataset/20210101_RM.csv')

In [4]:
# Mostramos lo que tiene
df.head()

,PHONE_ID,timestamp,bts_id,lat,lon
0,668f7c17a62c937a75f762c7198a7fc98ed4e0e0c64ce0...,2021-01-01T17:22:55.000-03:00,CEMG1,-33.3913,-70.6222
1,780fc36e9a2bc99de12adb740e5e82b3cabba75c1ecd23...,2021-01-01T17:10:19.000-03:00,CEMG1,-33.3913,-70.6222
2,b3c52936d4f8494dae9d1158ce76951e62413d511f5fe2...,2021-01-01T00:35:04.000-03:00,CEMG1,-33.3913,-70.6222
3,a4ab622fe4c0de513c389ab475cee4ad5b5d27e07e32d9...,2021-01-01T17:22:38.000-03:00,CEMG1,-33.3913,-70.6222
4,77d8edaa34e7ac318ef33541957e9f33826dff24217636...,2021-01-01T17:11:12.000-03:00,CEMG1,-33.3913,-70.6222


In [5]:
# Nos deshacemos de Phone_ID y Timestamp, no los necesitamos
df = df.drop(columns = ['PHONE_ID', 'timestamp'])

In [6]:
# Ordenamos las antenas por su id
df = df.groupby(['bts_id']).mean()

In [7]:
# Obbservamos que ahora tenemos las antenas ordenadas y de forma única, sin datos repetidos
df

,lat,lon
bts_id,,
11SEP,-33.422300,-70.6111
14COS,-33.543100,-70.6860
15COS,-33.442500,-70.7589
2NENT,-33.537200,-70.6198
4CENF,-33.415100,-70.5620
...,...,...
WMQUI,-33.361000,-70.7013
WSMTF,-33.408900,-70.5679
YELCH,-33.471833,-70.7117


In [8]:
# Para leer los datos con Geopy, es necesario cambiar la latitud y longitud a strings
df['lat'] = df['lat'].astype("string")
df['lon'] = df['lon'].astype("string")

# Ahora, creamos una nueva columna con la latitud y longitud juntas
df['coordinates'] = df['lat'] + ',' + df['lon']

In [9]:
# Establecemos nuestro userAgent de Nominatim para hacer peticiones al servidor
geolocator = Nominatim(user_agent="analisis_comunas")

# Creamos una nueva columna y obtenemos la dirección respecto a las
# coordenadas obtenidas por cada antena disponible.
df['direction'] = df['coordinates'].apply(geolocator.reverse)
df

,lat,lon,coordinates,direction
bts_id,,,,
11SEP,-33.4223,-70.6111,"-33.4223,-70.6111","(Edificio Florencia, Andrés de Fuenzalida, San..."
14COS,-33.5431,-70.686,"-33.5431,-70.686","(Avenida José Joaquín Prieto Vial, Villa Las M..."
15COS,-33.4425,-70.7589,"-33.4425,-70.7589","(8949, Avenida San Pablo, Pudahuel, Provincia ..."
2NENT,-33.5372,-70.6198,"-33.5372,-70.6198","(Mercado Modelo San Gregorio, 8286, Avenida Ca..."
4CENF,-33.4151,-70.562,"-33.4151,-70.562","(6371, Martín de Zamora, Las Condes, Provincia..."
...,...,...,...,...
WMQUI,-33.361,-70.7013,"-33.361,-70.7013","(Walmart Chile, 8301, Avenida Presidente Eduar..."
WSMTF,-33.4089,-70.5679,"-33.4089,-70.5679","(Noreste, Avenida Apoquindo, Nueva Las Condes,..."
YELCH,-33.471832727154215,-70.7117,"-33.471832727154215,-70.7117","(Licorería Aeropuerto, Yelcho, Villa Francia, ..."


In [10]:
# Ahora, creamos una nueva columna y le asignamos las direcciones como objetos
# de esta manera, así podremos separar las comunas con un listado.
df['direc'] = df['direction'].apply(lambda x: x.address)
df

,lat,lon,coordinates,direction,direc
bts_id,,,,,
11SEP,-33.4223,-70.6111,"-33.4223,-70.6111","(Edificio Florencia, Andrés de Fuenzalida, San...","Edificio Florencia, Andrés de Fuenzalida, Sanh..."
14COS,-33.5431,-70.686,"-33.5431,-70.686","(Avenida José Joaquín Prieto Vial, Villa Las M...","Avenida José Joaquín Prieto Vial, Villa Las Ma..."
15COS,-33.4425,-70.7589,"-33.4425,-70.7589","(8949, Avenida San Pablo, Pudahuel, Provincia ...","8949, Avenida San Pablo, Pudahuel, Provincia d..."
2NENT,-33.5372,-70.6198,"-33.5372,-70.6198","(Mercado Modelo San Gregorio, 8286, Avenida Ca...","Mercado Modelo San Gregorio, 8286, Avenida Car..."
4CENF,-33.4151,-70.562,"-33.4151,-70.562","(6371, Martín de Zamora, Las Condes, Provincia...","6371, Martín de Zamora, Las Condes, Provincia ..."
...,...,...,...,...,...
WMQUI,-33.361,-70.7013,"-33.361,-70.7013","(Walmart Chile, 8301, Avenida Presidente Eduar...","Walmart Chile, 8301, Avenida Presidente Eduard..."
WSMTF,-33.4089,-70.5679,"-33.4089,-70.5679","(Noreste, Avenida Apoquindo, Nueva Las Condes,...","Noreste, Avenida Apoquindo, Nueva Las Condes, ..."
YELCH,-33.471832727154215,-70.7117,"-33.471832727154215,-70.7117","(Licorería Aeropuerto, Yelcho, Villa Francia, ...","Licorería Aeropuerto, Yelcho, Villa Francia, E..."


In [11]:
# Con esta función, se busca la comuna dentro de la dirección y retornamos
# sólo el valor en caso de que coincida.
def establecer_comuna(x):
  # Comunas de chile:
  comunas = ['Peñalolén', 'Quinta Normal', 'María Pinto', 'Maipú', 'Lo Espejo', 'La Reina', 'San Joaquín', 'Independencia', 
            'San José de Maipo', 'Peñaflor', 'Huechuraba', 'Talagante', 'Paine', 'Colina', 'Pedro Aguirre Cerda', 
            'Cerro Navia', 'Cerrillos', 'Lo Prado', 'Santiago', 'La Cisterna', 'El Bosque', 'Estación Central', 'La Pintana', 
            'Lo Barnechea', 'Pudahuel', 'Las Condes', 'Vitacura', 'Tiltil', 'San Pedro', 'Buin', 'Quilicura', 'Padre Hurtado', 
            'Pirque', 'El Monte', 'Recoleta', 'La Granja', 'San Ramón', 'Renca', 'Conchalí', 'Melipilla', 'Alhué', 'Ñuñoa', 
            'San Miguel', 'Curacaví', 'Calera de Tango', 'Lampa', 'Puente Alto', 'Providencia', 'Macul', 'Isla de Maipo', 
            'San Bernardo', 'La Florida']
  for comuna in comunas:
    found = x.find(comuna)
    if found != -1 or found != None:
      return comuna

In [12]:
# Aplicamos la función y guardamos todos los datos en "comuna"
df['comuna'] = df['direc'].apply(establecer_comuna)
# Revisamos ahora las comunas en un vistazo rápido
df

,lat,lon,coordinates,direction,direc,comuna
bts_id,,,,,,
11SEP,-33.4223,-70.6111,"-33.4223,-70.6111","(Edificio Florencia, Andrés de Fuenzalida, San...","Edificio Florencia, Andrés de Fuenzalida, Sanh...",Peñalolén
14COS,-33.5431,-70.686,"-33.5431,-70.686","(Avenida José Joaquín Prieto Vial, Villa Las M...","Avenida José Joaquín Prieto Vial, Villa Las Ma...",Peñalolén
15COS,-33.4425,-70.7589,"-33.4425,-70.7589","(8949, Avenida San Pablo, Pudahuel, Provincia ...","8949, Avenida San Pablo, Pudahuel, Provincia d...",Peñalolén
2NENT,-33.5372,-70.6198,"-33.5372,-70.6198","(Mercado Modelo San Gregorio, 8286, Avenida Ca...","Mercado Modelo San Gregorio, 8286, Avenida Car...",Peñalolén
4CENF,-33.4151,-70.562,"-33.4151,-70.562","(6371, Martín de Zamora, Las Condes, Provincia...","6371, Martín de Zamora, Las Condes, Provincia ...",Peñalolén
...,...,...,...,...,...,...
WMQUI,-33.361,-70.7013,"-33.361,-70.7013","(Walmart Chile, 8301, Avenida Presidente Eduar...","Walmart Chile, 8301, Avenida Presidente Eduard...",Peñalolén
WSMTF,-33.4089,-70.5679,"-33.4089,-70.5679","(Noreste, Avenida Apoquindo, Nueva Las Condes,...","Noreste, Avenida Apoquindo, Nueva Las Condes, ...",Peñalolén
YELCH,-33.471832727154215,-70.7117,"-33.471832727154215,-70.7117","(Licorería Aeropuerto, Yelcho, Villa Francia, ...","Licorería Aeropuerto, Yelcho, Villa Francia, E...",Peñalolén


In [13]:
# Como en la lista hay 52 comunas y existen aquí 53, es probable
# que tengamos algún dato que Open Street Map no haya podido
# encontrar de forma correcta y no se haya asignado.
print("Número de elementos únicos:", len(df['comuna'].unique()))
print("Todas las comunas:\n", df['comuna'].unique())

Número de elementos únicos: 1
Todas las comunas:
 ['Peñalolén']


In [14]:
# Ya que hay una comuna no identificada, buscamos dónde está para
# asignarla manualmente con datos de alguna otra plataforma

# df['comuna'].index.get_indexer([None])

# df[df.eq(None).any(axis=1)]
def getNull(df):
    cont = 0
    contn = 0
    for i in df:
        if i == None:
            cont += 1
            print(contn)
        contn += 1
    return(cont)

In [15]:
getNull(df['comuna'])

0

In [16]:
# Establecemos la comuna de la antena 103 como Paine
# de forma manual, de acuerdo a Microsoft Bing Maps.
# Angostura Paine, Región Metropolitana, Chile
# www.bing.com/maps?osid=52f8302d-62ce-48d9-9bc5-9ea6bfc0db01&cp=-33.915089~-70.729487
df['comuna'][103] = 'Paine'

/tmp/ipykernel_586920/2142168937.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comuna'][103] = 'Paine'


In [17]:
# Eliminamos las columnas que no vamos a necesitar más adelante
df = df.drop(columns = ['direction', 'direc', 'lat', 'lon']) # lat y lon añadidos recientemente!
df

,coordinates,comuna
bts_id,,
11SEP,"-33.4223,-70.6111",Peñalolén
14COS,"-33.5431,-70.686",Peñalolén
15COS,"-33.4425,-70.7589",Peñalolén
2NENT,"-33.5372,-70.6198",Peñalolén
4CENF,"-33.4151,-70.562",Peñalolén
...,...,...
WMQUI,"-33.361,-70.7013",Peñalolén
WSMTF,"-33.4089,-70.5679",Peñalolén
YELCH,"-33.471832727154215,-70.7117",Peñalolén


In [18]:
# Guardamos el dataset para hacerle merge más adelante
df.to_csv('../dataset/comunas.csv')